In [3]:
import os

from openai import OpenAI

OPENAI_API_KEY = os.environ['OPENAI_API_KEY']

client = OpenAI(api_key=OPENAI_API_KEY)

completion = client.chat.completions.create(
  model='gpt-3.5-turbo-0125',
  messages=[{'role': 'user', 'content': 'hi'}],
  temperature=0.0,
)

print(completion.choices[0].message.content)



Hello! How can I assist you today?


In [4]:
import json

with open('./res/reviews.json') as f:
    review_list = json.load(f)


review_list[:3]

[{'review': '위치는 최상!!!  주차는 ㅠㅠ', 'stars': 4, 'date': '1일 전'},
 {'review': '호텔 위치가 안녕인사동 건물이라 찾기 쉽고 여행하기 너무 최적의 장소였어요! 그 건물 미피전시회 보러왔다가 너무 편하게 놀다가네요. 호텔 조식 좋아하는데 나쁘진않치만 근처에 먹을데가 너무 많아서 다음엔 신청 안할려구용 ㅋ 침대가 엄청 편하고 호텔이 너무 깨끗했습니다. 체크인이 오래걸리는 문제만 해결되면 재방문 의사있습니다!',
  'stars': 5,
  'date': '2일 전'},
 {'review': '위치 굿 \n객실내 먼지가 좀 많고 건조함 \n뷰는 크게 기대하지 마세요',
  'stars': 4,
  'date': '2일 전'}]

In [5]:
good_cnt, bad_cnt = 0, 0

for r in review_list:
  if r['stars'] == 5:
    good_cnt += 1
  else:
    bad_cnt += 1

good_cnt, bad_cnt

(99, 21)

In [7]:
reviews_good, reviews_bad = [], []

for r in review_list:
  if r['stars'] == 5:
    reviews_good.append('[REVIEW_START]' + r['review'] + '[REVIEW_END]')
  else:
    reviews_bad.append('[REVIEW_START]' + r['review'] + '[REVIEW_END]')

reviews_bad[:3]

['[REVIEW_START]위치는 최상!!!  주차는 ㅠㅠ[REVIEW_END]',
 '[REVIEW_START]위치 굿 \n객실내 먼지가 좀 많고 건조함 \n뷰는 크게 기대하지 마세요[REVIEW_END]',
 '[REVIEW_START]너무 번화가 쪽은 아니라서 밤에 조용히 쉬기 좋았어요[REVIEW_END]']

In [8]:
reviews_good_text = '\n'.join(reviews_good)
reviews_bad_text = '\n'.join(reviews_bad)

reviews_bad_text[:100]

'[REVIEW_START]위치는 최상!!!  주차는 ㅠㅠ[REVIEW_END]\n[REVIEW_START]위치 굿 \n객실내 먼지가 좀 많고 건조함 \n뷰는 크게 기대하지 마세요[REV'

In [9]:
import datetime
from dateutil import parser

def preprocess_reviews(path='./res/reviews.json'):
  with open(path) as f:
    review_list = json.load(f)

  reviews_good, reviews_bad = [], []

  current_date = datetime.datetime.now()
  date_boundary = current_date - datetime.timedelta(days=6*30)

  for r in review_list:
    review_date = r['date']
    try :
      review_date = parser.parse(review_date)
    except (ValueError, TypeError):
      review_date = current_date

    if review_date < date_boundary:
      continue

    if r['stars'] == 5:
      reviews_good.append('[REVIEW_START]' + r['review'] + '[REVIEW_END]')
    else:
      reviews_bad.append('[REVIEW_START]' + r['review'] + '[REVIEW_END]')

  reviews_good_text = '\n'.join(reviews_good)
  reviews_bad_text = '\n'.join(reviews_bad)

  return reviews_good_text, reviews_bad_text


good, bad = preprocess_reviews()

good[:100]

'[REVIEW_START]호텔 위치가 안녕인사동 건물이라 찾기 쉽고 여행하기 너무 최적의 장소였어요! 그 건물 미피전시회 보러왔다가 너무 편하게 놀다가네요. 호텔 조식 좋아하는데 '

# 평가용 함수 작성

In [10]:
def pairwise_eval(reviews, answer_a, answer_b):
  eval_prompt = f"""[System]
Please act as an impartial judge and evaluate the quality of the Korean summaries provided by two AI assistants to the set of user reviews on accommodations displayed below. You should choose the assistant that follows the user's instructions and answers the user's questions better. Your evaluation should consider factors such as the helpfulness, relevance, accuracy, depth, creativity, and level of detail of their responses. You should focus on who provides a better answer to the second user question. Begin your evaluation by comparing the responses of the two assistants and provide a short explanation. Avoid any position biases and ensure that the order in which the responses were presented does not influence your decision. Do not allow the length of the responses to influence your evaluation. Do not favor certain names of the assistants. Be as objective as possible. After providing your explanation, output your final verdict by strictly following this format: \"[[A]]\" if assistant A is better, \"[[B]]\" if assistant B is better, and \"[[C]]\" for a tie.

[User Reviews]
{reviews}
[The Start of Assistant A's Answer]
{answer_a}
[The End of Assistant A's Answer]
[The Start of Assistant B's Answer]
{answer_b}
[The End of Assistant B's Answer]"""
  
  completion = client.chat.completions.create(
    model='gpt-4o-2024-05-13',
    messages=[{'role': 'system', 'content': eval_prompt}],
    temperature=0.0,
  )

  return completion

## Baseline 모델 개발

In [11]:
PROMPT_BASELINE = f"""아래 숙소 리뷰에 대해 5문장 내로 요약해줘:"""

In [ ]:
reviews, _ = preprocess_reviews(path='./res/reviews.json')

def summarize(reviews, prompt, temperature=0.0, model='gpt-3.5-turbo-0125'):
  prompt = prompt + '\n\n' + reviews

  completion = client.chat.completions.create(
    model=model,
    messages=[{'role': 'system', 'content': prompt}],
    temperature=temperature,
  )

  return completion

print(summarize(reviews, PROMPT_BASELINE).choices[0].message.content)